In [96]:
import cv2
import numpy as np

In [65]:
F0 = cv2.imread("../Images/sim_Landsat_t1.tif")
C0 = cv2.imread("../Images/sim_MODIS_t1.tif")
C1 = cv2.imread("../Images/sim_MODIS_t2.tif")

In [66]:
type(F0)

numpy.ndarray

In [67]:
F0.shape

(150, 150, 3)

In [68]:
windowSize = 3
padAmount = windowSize//2
print(padAmount)

1


In [69]:
padded_F0 = np.pad(F0, [(padAmount, padAmount), (padAmount, padAmount), (0, 0)], mode='constant', constant_values=0)

In [70]:
padded_F0.shape

(152, 152, 3)

In [71]:
F0_test = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])

In [72]:
def padArray(arr, windowSize):
    padAmount = windowSize//2
    return np.pad(arr, [(padAmount, padAmount), (padAmount, padAmount), (0, 0)], mode='constant', constant_values=0)

In [73]:
F0_test.shape

(2, 2, 3)

In [74]:
def printArray(arr):
    for row in arr:
        for e in row:
            print(e, end=" ")
        print("\n")

In [75]:
printArray(F0_test)

[1 2 3] [4 5 6] 

[7 8 9] [10 11 12] 



In [76]:
F0_test.shape

(2, 2, 3)

In [77]:
np.set_printoptions(linewidth=np.inf)

In [78]:
padded_F0_test = padArray(F0_test, 3)

In [79]:
padded_F0_test.shape

(4, 4, 3)

In [80]:
printArray(padded_F0_test)

[0 0 0] [0 0 0] [0 0 0] [0 0 0] 

[0 0 0] [1 2 3] [4 5 6] [0 0 0] 

[0 0 0] [7 8 9] [10 11 12] [0 0 0] 

[0 0 0] [0 0 0] [0 0 0] [0 0 0] 



In [85]:
print(padAmount)

1


In [94]:
i, j = 1, 1
window = padded_F0_test[i-windowSize//2:i+windowSize//2+1, j-windowSize//2:j+windowSize//2+1]
printArray(window)

[0 0 0] [0 0 0] [0 0 0] 

[0 0 0] [1 2 3] [4 5 6] 

[0 0 0] [7 8 9] [10 11 12] 



numpy.ndarray

In [95]:
for i in range(padAmount, len(padded_F0_test)-padAmount):
    for j in range(padAmount, len(padded_F0_test[0])-padAmount):
        window = padded_F0_test[i-windowSize//2:i+windowSize//2+1, j-windowSize//2:j+windowSize//2+1]
        printArray(window)
        print("NEXT WINDOW")
    

[0 0 0] [0 0 0] [0 0 0] 

[0 0 0] [1 2 3] [4 5 6] 

[0 0 0] [7 8 9] [10 11 12] 

NEXT WINDOW
[0 0 0] [0 0 0] [0 0 0] 

[1 2 3] [4 5 6] [0 0 0] 

[7 8 9] [10 11 12] [0 0 0] 

NEXT WINDOW
[0 0 0] [1 2 3] [4 5 6] 

[0 0 0] [7 8 9] [10 11 12] 

[0 0 0] [0 0 0] [0 0 0] 

NEXT WINDOW
[1 2 3] [4 5 6] [0 0 0] 

[7 8 9] [10 11 12] [0 0 0] 

[0 0 0] [0 0 0] [0 0 0] 

NEXT WINDOW


# TESTING SPECTRAL DISTANCE

In [97]:
import cv2
import numpy as np

In [98]:
windowSize = 3
padAmount = windowSize//2

### Padding array

In [100]:
def padArray(arr, windowSize):
    padAmount = windowSize//2
    return np.pad(arr, [(padAmount, padAmount), (padAmount, padAmount), (0, 0)], mode='constant', constant_values=0)

### Print cell

In [105]:
def printArray(arr):
    for row in arr:
        for e in row:
            print(e, end=" ")
        print("\n")

In [106]:
F0_test = np.array([[[1, 2, 3], [4, 5, 6]], [[7, 8, 9], [10, 11, 12]]])
padded_F0_test = padArray(F0_test, windowSize)

In [113]:
C0_test = np.array([[[1, 1, 1], [1, 1, 1]], [[1, 1, 1], [1, 1, 1]]])
padded_C0_test = padArray(C0_test, windowSize)

### Spectral Distance

In [117]:
for i in range(padAmount, len(padded_F0_test)-padAmount):
    for j in range(padAmount, len(padded_F0_test[0])-padAmount):
        F0_window = padded_F0_test[i-windowSize//2:i+windowSize//2+1, j-windowSize//2:j+windowSize//2+1]
        C0_window = padded_C0_test[i-windowSize//2:i+windowSize//2+1, j-windowSize//2:j+windowSize//2+1]
        print(f"Window around point ({i}, {j})")
        spectral_difference_window = F0_window - C0_window
        spectral_distance_window = 1/(abs(spectral_difference_window) + 1.0)
        

Window around point (1, 1)
[1. 1. 1.] [1. 1. 1.] [1. 1. 1.] 

[1. 1. 1.] [1.         0.5        0.33333333] [0.25       0.2        0.16666667] 

[1. 1. 1.] [0.14285714 0.125      0.11111111] [0.1        0.09090909 0.08333333] 

Window around point (1, 2)
[1. 1. 1.] [1. 1. 1.] [1. 1. 1.] 

[1.         0.5        0.33333333] [0.25       0.2        0.16666667] [1. 1. 1.] 

[0.14285714 0.125      0.11111111] [0.1        0.09090909 0.08333333] [1. 1. 1.] 

Window around point (2, 1)
[1. 1. 1.] [1.         0.5        0.33333333] [0.25       0.2        0.16666667] 

[1. 1. 1.] [0.14285714 0.125      0.11111111] [0.1        0.09090909 0.08333333] 

[1. 1. 1.] [1. 1. 1.] [1. 1. 1.] 

Window around point (2, 2)
[1.         0.5        0.33333333] [0.25       0.2        0.16666667] [1. 1. 1.] 

[0.14285714 0.125      0.11111111] [0.1        0.09090909 0.08333333] [1. 1. 1.] 

[1. 1. 1.] [1. 1. 1.] [1. 1. 1.] 

